In [3]:
import json

In [25]:
loaded_json = {
    "1": {
        '1.1': "some text about features",
        '1.2': "new text",
          },
    "2": {
        '2.1': 'few text',
        '2.2': 'ok',
    }
}

In [26]:
print(type(loaded_json) == type(dict()))

True


In [59]:
def is_word_in_text(word, text):
    word = word.lower()
    text_splited = text.lower().split()
    return (word in text_splited) or (word.split()[0] in text_splited)


finded_paths = []
def find_word_in_json(loaded_json, word, path=""):
  global finded_paths
  if path == "":
    finded_paths = []
  for key, item in loaded_json.items():
    if isinstance(item, type(dict())):
      find_word_in_json(item, word, path + key + '/')
    else:
      if is_word_in_text(word, item):
        finded_paths.append(path + key + '/')
  return finded_paths
texts = ''
def find_texts_in_json(loaded_json, word, path=""):
  global texts
  if path == "":
    texts = ''
  for key, item in loaded_json.items():
    if isinstance(item, type(dict())):
      find_texts_in_json(item, word, path + key + '/')
    else:
      if is_word_in_text(word, item):
        texts += item + '\n'
  return texts

In [61]:
# testing find_word_in_json func
print(find_word_in_json(loaded_json, 'text'))
print(find_texts_in_json(loaded_json, 'text'))

['1/1.1/', '1/1.2/', '2/2.1/']
some text about features
new text
few text



## Uploading llm model

In [81]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="openai-community/gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [82]:
print(find_texts_in_json(loaded_json, 'text'))

some text about features
new text
few text



In [83]:
pipe(find_texts_in_json(loaded_json, 'text'))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "some text about features\nnew text\nfew text\na few screenshots of other features (note that there are lots of them) but we'd like to make it clear which one we're doing\nsome text about features\nnew text\nsome text"}]

In [85]:
loaded_json = {
    "1": {
        '1.1': "some text about features",
        '1.2': "new text",
          },
    "2": {
        '2.1': 'few text',
        '2.2': 'ok',
    }
}

In [88]:
# Example of working our final model -)
# llm promt format: (List of requirements in reglament, promt part, info from UseCase)

pipe("speed of car cant be more than 3 km per hour " +
    "We invented new type of car that accelerates to 5 km per hour" +
     "check conformance requirement with next text (if not, return detailed comments according to requirement) : ")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'speed of car cant be more than 3 km per hour We invented new type of car that accelerates to 5 km per hour.check conformance requirement with next text (if not, return detailed comments according to requirement) : -------------- next part '}]

In [76]:
model_answers = ""
UseCaseText = "sentence shouldnt start with s" # here is a text description of your project
finded_reglaments_names = ['AVAS', 'Braking system', 'EMC']
finded_reglament_words = ["Doors", "Steering mechanism", "Safety belt", 'text']
for reglament_name in finded_reglaments_names[:1]:
  # reglament_en = json.loads('path/to/reglament.json')
  reglament_en = loaded_json
  for word in finded_reglament_words:
    founded_texts = find_texts_in_json(reglament_en, word)
    # print(word)
    if founded_texts != '':
      model_answers += pipe((find_texts_in_json(reglament_en, word) + f' check this requirements related to word {word} in this text:' + UseCaseText))[0]['generated_text']

print(model_answers)

check this requirements related to word text
